In [8]:
#import libraries
from googletrans import Translator
translator = Translator()
from googletrans import LANGUAGES
import pyodbc 
import json

In [9]:
#connect to db
server_name = "INTERN-LAPTOP\MSSQLSERVER02"
database_name = "iabcategoriesdb"

connection_string = f'DRIVER={{ODBC Driver 18 for SQL Server}};SERVER={server_name};DATABASE={database_name};Trusted_Connection=yes;TrustServerCertificate=yes'

connection = pyodbc.connect(connection_string)
cursor = connection.cursor()

In [10]:
#grab new language(s) from scraped website
with open("../scrapeContent/scraped_content_output.json", "r") as jsonobj:
    scraped_json = json.load(jsonobj)
    
all_source_languages = scraped_json["all_source_languages"]

cursor.execute("SELECT DISTINCT keyword_language FROM keywords")
db_keyword_languages_result = cursor.fetchall()
db_keyword_languages = set([row[0] for row in db_keyword_languages_result])


new_languages = [language for language in all_source_languages if language not in db_keyword_languages]

In [15]:
#verify if the new_language have valid LANGUAGE code for translation
keyword_new_language_array = [] 

for language in new_languages:
    if language not in LANGUAGES:
        print(f"'{language}' is not a valid language code.")
    else:
        keyword_new_language_array.append(language)

[]


In [12]:
#grab initial keywords, en, from database
if keyword_new_language_array:
    keyword_original_language = "en"

    cursor.execute(f"SELECT keyword FROM keywords WHERE keyword_language = ?", keyword_original_language)
    original_keywords_array = [row[0] for row in cursor.fetchall()]


In [13]:
if keyword_new_language_array:
    #translate keywords
    for keyword_new_language in keyword_new_language_array:
        translation_keywords_array = [(original_keyword, translator.translate(original_keyword, dest=keyword_new_language).text) for original_keyword in original_keywords_array]

    #store keywords and associate tables
    for translation_keyword in translation_keywords_array:
        try:
            cursor.execute("""
                SELECT ic.category_id 
                FROM keywords k 
                JOIN iabCategories_keywords ic ON k.keyword_id = ic.keyword_id 
                WHERE k.keyword = ?
                """, (translation_keyword[0],))
            
            category_id_result = cursor.fetchone()
            category_id = category_id_result[0]

            cursor.execute("INSERT INTO keywords (keyword, keyword_language) VALUES (?, ?)", translation_keyword[1], keyword_new_language)
            connection.commit()

            cursor.execute("SELECT @@IDENTITY AS last_id")
            last_id_result = cursor.fetchone()
            last_keyword_id = last_id_result[0]

            cursor.execute("INSERT INTO iabCategories_keywords (category_id, keyword_id) VALUES (?, ?)", (category_id, last_keyword_id))
            connection.commit()

        except Exception as e:
            print(f"The keyword(s) '{translation_keyword[1]}' already exist in the database, in relation to the '{keyword_new_language}' language.")

In [14]:
#close db connection
connection.close()